In [1]:
import os
import sys
import subprocess
import time
import pickle
import numpy as np
import cupy as cp

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (18,12)
mpl.rcParams['axes.grid'] = False
sns.set_style('whitegrid')

from sklearn.metrics import mean_squared_error

In [2]:
# Simulation variables
iter = 100 # Number of iterations
num_samples = 640 # Number of sequences to generate
dof = 10

rng = cp.random.default_rng()

# Arrays to store results
results_x = cp.empty((2*dof,num_samples*iter+num_samples))
results_x_hat = cp.zeros((2*dof,num_samples*iter+num_samples))
results_y = cp.zeros((dof,num_samples*iter+num_samples))
results_P = cp.zeros(num_samples*iter+num_samples)
results_est_err = cp.zeros((2*dof,num_samples*iter+num_samples))
results_m_err = cp.zeros((dof,num_samples*iter+num_samples))

In [3]:
# Discretize LTI system
dt = 0.01
subprocess.check_output([sys.executable, "cont2discrete_helper.py",f"{dof}",f"{dt}"])
with open('d_system.pickle', 'rb') as f:
     d_system = pickle.load(f)

A = cp.asarray(d_system[0]) # Process matrix
B = cp.asarray(d_system[1]) # Input gain
H = cp.asarray(d_system[2]) # Measurement matrix

In [4]:
# Noise parameters
sigma_p = 0.1 # Standard deviation of process noise
sigma_p_diag = (sigma_p**2)*cp.ones(2*dof)
Q = cp.diag(sigma_p_diag)

sigma_m = 0.25 # Standard deviation of measurement noise
sigma_m_diag = (sigma_m**2)*cp.ones(dof)
R = cp.diag(sigma_m_diag)

In [5]:
# Simulation
kf_time = 0

for b in range(num_samples):
    P = cp.identity(2*dof) # Initial covariance matrix for estimation error
    x_hat = cp.zeros((2*dof,1)) # Initial state estimate
    x = cp.random.multivariate_normal(cp.zeros(2*dof), Q).reshape(-1,1) # Initial true state

    # Store initial values
    results_x[:,b*(iter+1):b*(iter+1)+1] = x
    results_x_hat[:,b*(iter+1):b*(iter+1)+1] = x_hat
    results_y[:,b*(iter+1):b*(iter+1)+1] = cp.zeros((dof,1))
    results_P[b*(iter+1):b*(iter+1)+1] = cp.trace(P)

    # Record initial values for measurement
    v_0 = cp.random.multivariate_normal(cp.zeros(dof),R).reshape(-1,1)
    
    y = H.dot(x) + v_0 # Initial measurement

    input_cov = cp.full(dof,1)
    input_cov_matrix = cp.diag(input_cov) # mean and standard deviation for white noise icputs
    U = cp.random.multivariate_normal(cp.zeros(dof), input_cov_matrix, iter)
    W = cp.random.multivariate_normal(cp.zeros(2*dof), Q, iter)
    V = cp.random.multivariate_normal(cp.zeros(dof), R, iter)

    results_m_err[:,b+(iter+1):b*(iter+1)+1] = x[0::2]-y # Record initial measurement error
    results_y[:,b*(iter+1):b*(iter+1)+1] = y # Record initial measurement
    
    for t in range(1,iter+1):
        w = W[t-1].reshape(-1,1) # Process noise at time t
        v = V[t-1].reshape(-1,1) # Measurement noise at time t
        u = U[t-1].reshape(-1,1) #cp.sin(2*cp.pi*freq*t) - cp.array([x[0],x[2],x[4],x[6],x[8],x[10]]).reshape(-1,1)

        x = A.dot(x)+B.dot(u) + w # State at time t
        y = H.dot(x) + v # Measurement at time t
        results_m_err[:,b*(iter+1)+t:b*(iter+1)+1] = x[0::2]-y # Store measurement error at time t

        st = time.process_time()
        # Predict
        x_hat = A.dot(x_hat)+B.dot(u) # Predict state estimate
        P = A.dot(P).dot(A.conj().transpose())+Q # Predict estimate covariance matrix

        # Update
        K = P.dot(H.conj().transpose()).dot(cp.linalg.inv(H.dot(P).dot(H.conj().transpose())+R)) # Get optimal Kalman gain
        x_hat = x_hat + K.dot(y - H.dot(x_hat)) # Update state estimate
        P = (cp.identity(2*dof) - K.dot(H)).dot(P).dot((cp.identity(2*dof) - K.dot(H)).conj().transpose())+K.dot(R).dot(K.conj().transpose()) # Update estimate covariance matrix
        et = time.process_time()

        kf_time += (et-st)
        results_x[:,b*(iter+1)+t:b*(iter+1)+t+1] = x # Store true state at time t
        results_x_hat[:,b*(iter+1)+t:b*(iter+1)+t+1] = x_hat # Store state estimate at time t
        results_y[:,b*(iter+1)+t:b*(iter+1)+t+1] = y # Store measurement at time t
        results_est_err[:,b*(iter+1)+t:b*(iter+1)+t+1] = x - x_hat # Store MSE between true state and predicted state at time t
        results_P[b*(iter+1)+t:b*(iter+1)+t+1] = cp.trace(P)

c:\Users\Aki\AppData\Local\Programs\Python\Python310\lib\site-packages\cupy\random\_distributions.py:476: FutureWarning: cupy.random.multivariate_normal is experimental. The interface can change in the future.
  _util.experimental('cupy.random.multivariate_normal')
c:\Users\Aki\AppData\Local\Programs\Python\Python310\lib\site-packages\cupy\random\_generator.py:337: FutureWarning: cupy.random.RandomState.multivariate_normal is experimental. The interface can change in the future.
  _util.experimental('cupy.random.RandomState.multivariate_normal')


In [6]:
'''
# Plot results
fig = plt.figure(figsize=(48,6*dof))

for i in range(dof):
    plt.subplot(2*dof+1, 2, 4*i+1)
    plt.plot(results_x[2*i,:],'tab:purple',label='True State')
    plt.plot(results_y[i,:],'.',label='Measurement')
    plt.plot(results_x_hat[2*i,:],'r',label='Kalman')
    plt.legend()
    plt.xlabel('Time')
    plt.ylabel(f"x{i+1}")

    plt.subplot(2*dof+1, 2, 4*i+2)
    plt.plot(results_est_err[2*i,:],'r',label='Kalman')
    plt.legend()
    plt.xlabel('Time')
    plt.ylabel(f"x{i+1} error")

    plt.subplot(2*dof+1, 2, 4*i+3)
    plt.plot(results_x[2*i+1,:],'tab:purple',label='True State')
    plt.plot(results_x_hat[2*i+1,:],'r',label='Kalman')
    plt.legend()
    plt.xlabel('Time')
    plt.ylabel(f"x'{i+1}")

    plt.subplot(2*dof+1, 2, 4*i+4)
    plt.plot(results_est_err[2*i+1,:],'r',label='Kalman')
    plt.legend()
    plt.xlabel('Time')
    plt.ylabel(f"x'{i+1} error")

plt.subplot(2*dof+1, 2, 4*dof+1)
plt.plot(results_P,'r',label='Kalman')
plt.xlabel('Time')
plt.ylabel('Tr(P)');
'''

'\n# Plot results\nfig = plt.figure(figsize=(48,6*dof))\n\nfor i in range(dof):\n    plt.subplot(2*dof+1, 2, 4*i+1)\n    plt.plot(results_x[2*i,:],\'tab:purple\',label=\'True State\')\n    plt.plot(results_y[i,:],\'.\',label=\'Measurement\')\n    plt.plot(results_x_hat[2*i,:],\'r\',label=\'Kalman\')\n    plt.legend()\n    plt.xlabel(\'Time\')\n    plt.ylabel(f"x{i+1}")\n\n    plt.subplot(2*dof+1, 2, 4*i+2)\n    plt.plot(results_est_err[2*i,:],\'r\',label=\'Kalman\')\n    plt.legend()\n    plt.xlabel(\'Time\')\n    plt.ylabel(f"x{i+1} error")\n\n    plt.subplot(2*dof+1, 2, 4*i+3)\n    plt.plot(results_x[2*i+1,:],\'tab:purple\',label=\'True State\')\n    plt.plot(results_x_hat[2*i+1,:],\'r\',label=\'Kalman\')\n    plt.legend()\n    plt.xlabel(\'Time\')\n    plt.ylabel(f"x\'{i+1}")\n\n    plt.subplot(2*dof+1, 2, 4*i+4)\n    plt.plot(results_est_err[2*i+1,:],\'r\',label=\'Kalman\')\n    plt.legend()\n    plt.xlabel(\'Time\')\n    plt.ylabel(f"x\'{i+1} error")\n\nplt.subplot(2*dof+1, 2, 4*d

In [7]:
sample_mse = 0
for i in range(2*dof):
    sample_mse = sample_mse + mean_squared_error(cp.asnumpy(results_x[:iter,i]),cp.asnumpy(results_x_hat[:iter,i]))

sample_mse_var = 0
for i in range(2*dof):
    sample_mse_var = sample_mse_var + cp.var(cp.asnumpy(results_est_err[:iter,i]))

trP = results_P[iter-1]

print(f"Sample MSE: {sample_mse}")
print(f"Sample MSE (Var): {sample_mse_var}")
print(f"Average theoretical MSE: {cp.mean(results_P)}")
print(f"KF test set time: {0.2*kf_time} ms")

Sample MSE: 24.58185915505308
Sample MSE (Var): 23.052680163929047
Average theoretical MSE: 55.83221082851931
KF test set time: 18.990625 ms


In [8]:
# Export data to CSV
x_export = cp.asnumpy(results_x.transpose())
x_hat_export = cp.asnumpy(results_x_hat.transpose())
y_export = cp.asnumpy(results_y.transpose())

x_export = np.delete(x_export, np.linspace(0,(num_samples-1)*(iter+1),num_samples,dtype=int), axis=0)
x_hat_export = np.delete(x_hat_export, np.linspace(0,(num_samples-1)*(iter+1),num_samples,dtype=int), axis=0)
y_export = np.delete(y_export, np.linspace(0,(num_samples-1)*(iter+1),num_samples,dtype=int), axis=0)

export = np.concatenate((y_export, x_export),axis=1)

data_header = "y1"
for i in range(2,dof+1):
    data_header = data_header + "," + f"y{i}"
for i in range(1,2*dof+1):
    data_header = data_header + "," + f"x{i}"

kalman_results_header = "x1_hat"
for i in range(2,2*dof+1):
    kalman_results_header = kalman_results_header + "," + f"x{i}_hat"

# [batch size]_[num batches]_[sequence length]
np.savetxt("Data/data_10dof_smd_independent_samples_32_20_100_cupy.csv", export, delimiter=',', header=data_header, comments="")
np.savetxt("Data/kalman_results_10dof_smd_independent_samples_32_20_100_cupy.csv", x_hat_export, delimiter=',', header=kalman_results_header, comments="")

os.remove('d_system.pickle')
